In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
arr_iupac_name = []
with open('database/Dict_segment.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_iupac_name = np.concatenate((arr_iupac_name, d[key]))

arr_cid = []
with open('database/Dict_segment_cid.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_cid = np.concatenate((arr_cid, d[key]))
  arr_cid = arr_cid.astype(np.uint8)

arr_iupac_cut = []
with open('database/Dict_segment_cut.json') as f1:
  d = json.load(f1)
  for key in d.keys():
    arr_iupac_cut = np.concatenate((arr_iupac_cut, d[key]))

In [3]:
import os
dir_iupac_name = 'database/iupac_name/'
dir_iupac_cut = 'database/iupac_cut/'
dir_cid = 'database/cid/'

for file in os.listdir(dir_iupac_name):
  with open(dir_iupac_name+file, 'r') as f:
    d = json.load(f)
  arr_iupac_name = np.concatenate((arr_iupac_name, d['values']))
print(arr_iupac_name.shape, arr_iupac_name[-2:])

for file in os.listdir(dir_iupac_cut):
  with open(dir_iupac_cut+file, 'r') as f:
    d = json.load(f)
  arr_iupac_cut = np.concatenate((arr_iupac_cut, d['values']))
print(arr_iupac_cut.shape, arr_iupac_cut[-2:])

for file in os.listdir(dir_cid):
  with open(dir_cid+file, 'r') as f:
    d = json.load(f)
  arr_cid = np.concatenate((arr_cid, d['values']))
print(arr_cid.shape, arr_cid[-2:])

(8200,) ['(2S)-2-[(4-hydroxyphenyl)methyl]-3-methylidenepent-4-enoic acid'
 '[4-(carboxymethyl)phenyl]oxidanium']
(8200,) ['(|2|S)|-|2|-|[|(|4|-|hydroxy|phen|yl|)|meth|yl|]|-|3|-|meth|yl|idene|pent|-|4|-|en|oic acid'
 '[|4|-|(|carboxy|meth|yl|)|phen|yl|]|oxidanium']
(8200,) [154546822 154550731]


In [4]:
df = pd.DataFrame(list(zip(arr_cid, arr_iupac_name, arr_iupac_cut)), columns=['cid', 'iupacname', 'label'])
df.drop_duplicates(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8193 entries, 0 to 8199
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cid        8193 non-null   int64 
 1   iupacname  8193 non-null   object
 2   label      8193 non-null   object
dtypes: int64(1), object(2)
memory usage: 256.0+ KB


In [6]:
df.tail()

,cid,iupacname,label
8195,154262256,1-(4-hydroxyphenyl)heptan-3-one,1|-|(|4|-|hydroxy|phen|yl|)|hept|an|-|3|-|one
8196,154265009,3-(4-hydroxyphenyl)-2-silyloxypropanoate,3|-|(|4|-|hydroxy|phen|yl|)|-|2|-|silyloxy|pro...
8197,154364721,8-(4-hydroxyphenyl)octanoate,8|-|(|4|-|hydroxy|phen|yl|)|oct|an|oate
8198,154546822,(2S)-2-[(4-hydroxyphenyl)methyl]-3-methylidene...,(|2|S)|-|2|-|[|(|4|-|hydroxy|phen|yl|)|meth|yl...
8199,154550731,[4-(carboxymethyl)phenyl]oxidanium,[|4|-|(|carboxy|meth|yl|)|phen|yl|]|oxidanium


In [7]:
from sklearn.model_selection import train_test_split

train_df, all_val_df = train_test_split(df, test_size = 0.3, shuffle = True)

val_df, test_df = train_test_split(all_val_df, test_size = 0.5, shuffle = True)

# Train 70, Val 15, Test 15

In [8]:
print(len(train_df), len(val_df), len(test_df))

5735 1229 1229


In [9]:
train_df.reset_index(inplace=True)
val_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [10]:
train_df['file_names'] = train_df['cid'].astype(str)+'.png'
val_df['file_names'] = val_df['cid'].astype(str)+'.png'
test_df['file_names'] = test_df['cid'].astype(str)+'.png'

In [11]:
train_df.to_csv('dataframe/df_train.csv', index=False)
val_df.to_csv('dataframe/df_val.csv', index=False)
test_df.to_csv('dataframe/df_test.csv', index=False)

In [12]:
 np.savetxt("cid.txt", arr_cid, fmt="%s")

In [13]:
pd.read_csv('dataframe/df_train.csv').head()

,index,cid,iupacname,label,file_names
0,5401,57240352,3-(3-chlorocyclopentyl)propanoic acid,3|-|(|3|-|chloro|cyclo|pent|yl|)|prop|an|oic acid,57240352.png
1,5245,54334288,2-cyano-3-[4-(ethylamino)phenyl]prop-2-enoic acid,2|-|cyano|-|3|-|[|4|-|(|eth|yl|amino|)|phen|yl...,54334288.png
2,2295,12497509,2-propan-2-yloxysulfanyloxypropane,2|-|prop|an|-|2|-|yl|oxy|sulfan|yl|oxy|prop|ane,12497509.png
3,2908,12335,1-iodopentane,1|-|iodo|pent|ane,12335.png
4,7052,134883755,(2Z)-2-(1-phenylethoxyimino)acetic acid,(|2|Z)|-|2|-|(|1|-|phen|yl|eth|oxyimino|)|acet...,134883755.png


# Make Tokenize Label


In [14]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [15]:
from nami.AI.kme_tokenize import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['iupacname'].values)

100%|██████████| 5735/5735 [03:10<00:00, 30.07it/s]


## Save tokenizer state

In [16]:
import json

with open('../Image_Captioning/tokenizer/index2word.json', 'w') as f1:
  json.dump(tokenizer.index2word, f1)

with open('../Image_Captioning/tokenizer/longest_sentences.json', 'w') as f2:
  json.dump(tokenizer.longest_sentences, f2)

with open('../Image_Captioning/tokenizer/num_words.json', 'w') as f3:
  json.dump(tokenizer.num_words, f3)

with open('../Image_Captioning/tokenizer/word2count.json', 'w') as f4:
  json.dump(tokenizer.word2count, f4)

with open('../Image_Captioning/tokenizer/word2index.json', 'w') as f5:
  json.dump(tokenizer.word2index, f5)

In [17]:
seq_train = tokenizer.text_to_sequences(train_df['iupacname'].values)
seq_val = tokenizer.text_to_sequences(val_df['iupacname'].values)
seq_test = tokenizer.text_to_sequences(test_df['iupacname'].values)

100%|██████████| 1229/1229 [00:40<00:00, 30.33it/s]


In [18]:
seq_train.shape

(5735, 67)

In [19]:
new_col = [f'label{i}' for i in range(1, seq_train.shape[1]+1)]

seq_train_df = pd.DataFrame(seq_train, columns=new_col)
seq_val_df = pd.DataFrame(seq_val, columns=new_col)
seq_test_df = pd.DataFrame(seq_test, columns=new_col)

seq_val_df.head()

,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,...,label58,label59,label60,label61,label62,label63,label64,label65,label66,label67
0,1,4,5,71,5,39,5,36,10,73,...,0,0,0,0,0,0,0,0,0,0
1,1,4,5,6,18,5,40,21,10,11,...,0,0,0,0,0,0,0,0,0,0
2,1,4,5,48,10,5,4,5,12,13,...,0,0,0,0,0,0,0,0,0,0
3,1,15,5,17,15,5,6,18,5,40,...,0,0,0,0,0,0,0,0,0,0
4,1,4,5,8,37,74,5,27,33,18,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_df_concat = pd.concat([train_df.drop(['index', 'label'], axis=1), seq_train_df], axis=1)
val_df_concat = pd.concat([val_df.drop(['index', 'label'], axis=1), seq_val_df], axis=1)
test_df_concat = pd.concat([test_df.drop(['index', 'label'], axis=1), seq_test_df], axis=1)

In [21]:
val_df_concat.head()

,cid,iupacname,file_names,label1,label2,label3,label4,label5,label6,label7,...,label58,label59,label60,label61,label62,label63,label64,label65,label66,label67
0,142150505,"3-fluoro-6-methylhepta-1,5-diene",142150505.png,1,4,5,71,5,39,5,...,0,0,0,0,0,0,0,0,0,0
1,135151766,3-(4-hydroxyphenyl)-2-iodo-2-[iodo(methyl)amin...,135151766.png,1,4,5,6,18,5,40,...,0,0,0,0,0,0,0,0,0,0
2,141954814,3-butyl-3-propan-2-yldioxirane,141954814.png,1,4,5,48,10,5,4,...,0,0,0,0,0,0,0,0,0,0
3,150902331,2-[2-(4-hydroxyphenyl)ethyl]-4-methylpentanoic...,150902331.png,1,15,5,17,15,5,6,...,0,0,0,0,0,0,0,0,0,0
4,54225191,"3-cyclohexa-1,4-dien-1-yl-2-methoxyprop-2-enoi...",54225191.png,1,4,5,8,37,74,5,...,0,0,0,0,0,0,0,0,0,0


In [22]:
test_df_concat.head()

,cid,iupacname,file_names,label1,label2,label3,label4,label5,label6,label7,...,label58,label59,label60,label61,label62,label63,label64,label65,label66,label67
0,140977828,2-(4-hydroxyphenyl)-N-(methoxymethyl)acetamide,140977828.png,1,15,5,6,18,5,40,...,0,0,0,0,0,0,0,0,0,0
1,121370299,[2-bicyclo[2.2.1]heptanyl(chloro)methyl] hydro...,121370299.png,1,17,15,5,159,17,160,...,0,0,0,0,0,0,0,0,0,0
2,12992914,"(1S,2R)-2-cyclohexylcyclopropane-1-carboxylic ...",12992914.png,1,6,27,49,33,119,11,...,0,0,0,0,0,0,0,0,0,0
3,140975190,"2-(4,4-dichloro-2-methylcyclohexa-1,5-dien-1-y...",140975190.png,1,15,5,6,18,33,18,...,0,0,0,0,0,0,0,0,0,0
4,92171143,(4R)-4-methylhex-1-ene,92171143.png,1,6,18,57,5,18,5,...,0,0,0,0,0,0,0,0,0,0


In [23]:
train_df_concat.to_csv('dataframe/df_train_label.csv', index=False)
val_df_concat.to_csv('dataframe/df_val_label.csv', index=False)
test_df_concat.to_csv('dataframe/df_test_label.csv', index=False)